# Some important data transformation tools

## Multi Index, Hierarchical Indexing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
np.random.seed(1)
# manual multi-index creation:
# provide the index argument with a list of lists
data = pd.Series(np.random.randint(10, size = 9), 
                 index = [['a','a','a','b','b','b','c','c','c'],
                          [ 1 , 2 , 3 , 1 , 2 , 3 , 1 , 2 , 3 ]])

In [3]:
data

a  1    5
   2    8
   3    9
b  1    5
   2    0
   3    0
c  1    1
   2    7
   3    6
dtype: int32

In [4]:
data.index

MultiIndex(levels=[['a', 'b', 'c'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 1, 2, 2, 2], [0, 1, 2, 0, 1, 2, 0, 1, 2]])

In [5]:
# select via the outer index
data.loc['b']

1    5
2    0
3    0
dtype: int32

In [8]:
# select via the inner index
data.loc[:,2] 

a    8
b    0
c    7
dtype: int32

In [9]:
# the unstack function returns a new DataFrame where the values have been unstacked
# similar to tidyr's spread() function in R
data.unstack()

,1,2,3
a,5,8,9
b,5,0,0
c,1,7,6


In [10]:
# after unstacking, the index is no longer a multi index
data.unstack().index

Index(['a', 'b', 'c'], dtype='object')

In [11]:
data.unstack().shape

(3, 3)

In [12]:
# the inverse operation of unstack() is stack()
# applying both of these functions will return the same series
data.unstack().stack()

a  1    5
   2    8
   3    9
b  1    5
   2    0
   3    0
c  1    1
   2    7
   3    6
dtype: int32

In [13]:
# you can swap the levels of the multi index using swaplevel
data.swaplevel()

1  a    5
2  a    8
3  a    9
1  b    5
2  b    0
3  b    0
1  c    1
2  c    7
3  c    6
dtype: int32

In [17]:
# the .loc accessors work as expected
data.swaplevel().loc[:,'a']

1    5
2    8
3    9
dtype: int32

In [16]:
# swaplevel will keep the original order
# you may want to sort based on the new swapped index levels
# you must save the output as data remains unchanged
data.swaplevel().sort_index()

1  a    5
   b    5
   c    1
2  a    8
   b    0
   c    7
3  a    9
   b    0
   c    6
dtype: int32

In [18]:
print(data)

a  1    5
   2    8
   3    9
b  1    5
   2    0
   3    0
c  1    1
   2    7
   3    6
dtype: int32


In [19]:
# summing and other aggregate functions can be performed on an index-based level
# calling sum() on a series, will sum the whole series
data.sum()

41

In [20]:
# you can call sum on the level 0 (the first level of the index)
# we get sums for each value in the first level of the index
data.sum(level = 0)

a    22
b     5
c    14
dtype: int32

In [21]:
data.sum(level = 1)

1    11
2    15
3    15
dtype: int32

# Reshaping and Pivoting Data

In [22]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index  = pd.Index(['alpha', 'beta'], name='letter'),
                    columns= pd.Index(['one', 'two', 'three'], name = 'number'))
data

number,one,two,three
letter,,,
alpha,0,1,2
beta,3,4,5


In [23]:
data.stack()  # creates a multi-index

letter  number
alpha   one       0
        two       1
        three     2
beta    one       3
        two       4
        three     5
dtype: int32

In [24]:
data.stack().unstack()  # unstack undoes the creation of the stacks

number,one,two,three
letter,,,
alpha,0,1,2
beta,3,4,5


In [25]:
data.stack().unstack(0) # you can specify how the unstacking should be done
# here we specify that we should unstack the first level of the multi-index

letter,alpha,beta
number,,
one,0,3
two,1,4
three,2,5


In [26]:
data.stack().unstack('letter')
# you can specify the unstacking by the index level name

letter,alpha,beta
number,,
one,0,3
two,1,4
three,2,5


In [27]:
data.stack().unstack('number')

number,one,two,three
letter,,,
alpha,0,1,2
beta,3,4,5


### Unstacking can introduce missing values

In [31]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])  
# using the argument keys when concat series will produce a multi-index
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [35]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [36]:
data2.unstack().stack() # stack() will filter out missing values

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [37]:
data2.unstack().stack(dropna = False) # you can force stack to keep the NaNs

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

# Small example data wrangling

In [38]:
data = pd.read_csv('macrodata.csv')

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 14 columns):
year        203 non-null float64
quarter     203 non-null float64
realgdp     203 non-null float64
realcons    203 non-null float64
realinv     203 non-null float64
realgovt    203 non-null float64
realdpi     203 non-null float64
cpi         203 non-null float64
m1          203 non-null float64
tbilrate    203 non-null float64
unemp       203 non-null float64
pop         203 non-null float64
infl        203 non-null float64
realint     203 non-null float64
dtypes: float64(14)
memory usage: 22.3 KB


In [40]:
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


https://pandas.pydata.org/pandas-docs/stable/generated/pandas.PeriodIndex.html

In [41]:
# We can create a time based index of periods consisting of the year and quarter
periods = pd.PeriodIndex(year = data.year, quarter = data.quarter, name = 'date')

In [42]:
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')

In [43]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name = 'item')
columns

Index(['realgdp', 'infl', 'unemp'], dtype='object', name='item')

In [44]:
data = data.reindex(columns = columns)

In [47]:
periods.to_timestamp('D','end')  # changes 1959Q1 to a date: the end date of Q1 of 1959: 1959-03-31

DatetimeIndex(['1959-03-31', '1959-06-30', '1959-09-30', '1959-12-31',
               '1960-03-31', '1960-06-30', '1960-09-30', '1960-12-31',
               '1961-03-31', '1961-06-30',
               ...
               '2007-06-30', '2007-09-30', '2007-12-31', '2008-03-31',
               '2008-06-30', '2008-09-30', '2008-12-31', '2009-03-31',
               '2009-06-30', '2009-09-30'],
              dtype='datetime64[ns]', name='date', length=203, freq='Q-DEC')

In [48]:
data.index = periods.to_timestamp('D','end')

In [49]:
data.head()

item,realgdp,infl,unemp
date,,,
1959-03-31,2710.349,0.00,5.8
1959-06-30,2778.801,2.34,5.1
1959-09-30,2775.488,2.74,5.3
1959-12-31,2785.204,0.27,5.6
1960-03-31,2847.699,2.31,5.2


In [50]:
data.stack().head(10)  # stack creates a series

date        item   
1959-03-31  realgdp    2710.349
            infl          0.000
            unemp         5.800
1959-06-30  realgdp    2778.801
            infl          2.340
            unemp         5.100
1959-09-30  realgdp    2775.488
            infl          2.740
            unemp         5.300
1959-12-31  realgdp    2785.204
dtype: float64

In [53]:
data.stack().reset_index().head()
# calling reset index turns the current index into a new column and creates a new index

,date,item,0
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801
4,1959-06-30,infl,2.340


In [54]:
ldata = data.stack().reset_index().rename(columns = {0: 'value'})  
# rename changes the column title '0' to 'value'
ldata.head(10)

,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801
4,1959-06-30,infl,2.340
5,1959-06-30,unemp,5.100
6,1959-09-30,realgdp,2775.488
7,1959-09-30,infl,2.740
8,1959-09-30,unemp,5.300
9,1959-12-31,realgdp,2785.204


In [57]:
# unstack doesn't work, because the stacking and unstacking is powered by multi-index
ldata.unstack()

date   0      1959-03-31 00:00:00
       1      1959-03-31 00:00:00
       2      1959-03-31 00:00:00
       3      1959-06-30 00:00:00
       4      1959-06-30 00:00:00
       5      1959-06-30 00:00:00
       6      1959-09-30 00:00:00
       7      1959-09-30 00:00:00
       8      1959-09-30 00:00:00
       9      1959-12-31 00:00:00
       10     1959-12-31 00:00:00
       11     1959-12-31 00:00:00
       12     1960-03-31 00:00:00
       13     1960-03-31 00:00:00
       14     1960-03-31 00:00:00
       15     1960-06-30 00:00:00
       16     1960-06-30 00:00:00
       17     1960-06-30 00:00:00
       18     1960-09-30 00:00:00
       19     1960-09-30 00:00:00
       20     1960-09-30 00:00:00
       21     1960-12-31 00:00:00
       22     1960-12-31 00:00:00
       23     1960-12-31 00:00:00
       24     1961-03-31 00:00:00
       25     1961-03-31 00:00:00
       26     1961-03-31 00:00:00
       27     1961-06-30 00:00:00
       28     1961-06-30 00:00:00
       29     

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.pivot.html

In [58]:
# if the data is in 'long' form, you can change it to 'wide' form with pivot
ldata.pivot('date','item','value').head()

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2


In [59]:
data.head()

item,realgdp,infl,unemp
date,,,
1959-03-31,2710.349,0.00,5.8
1959-06-30,2778.801,2.34,5.1
1959-09-30,2775.488,2.74,5.3
1959-12-31,2785.204,0.27,5.6
1960-03-31,2847.699,2.31,5.2


# Group By


In [60]:
np.random.seed(1)
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randint(20, size = 5),
                   'data2' : np.random.randint(20, size = 5)})
df

,key1,key2,data1,data2
0,a,one,5,11
1,a,two,11,5
2,b,one,12,15
3,b,two,8,0
4,a,one,9,16


In [61]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [62]:
grouped.mean()

key1
a     8.333333
b    10.000000
Name: data1, dtype: float64

In [64]:
df.groupby(df['key1']).mean()  
# if you don't specify the column, it'll apply the function to the entire dataframe

,data1,data2
key1,,
a,8.333333,10.666667
b,10.000000,7.500000


In [66]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means
# means has a multi-index

key1  key2
a     one      7
      two     11
b     one     12
      two      8
Name: data1, dtype: int32

In [68]:
# with the multi-index, you can unstack
means.unstack()

key2,one,two
key1,,
a,7,11
b,12,8


In [71]:
# you can perform group by on Series that are not in the dataframe, but are of the correct length
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    11.0
            2006    12.0
Ohio        2005     6.5
            2006     9.0
Name: data1, dtype: float64

In [73]:
# groupby applied to the entire dataframe, not just one column
df.groupby('key1').mean()
df.groupby(['key1', 'key2']).mean()

data1  data2
key1 key2              
a    one     7.0   13.5
     two    11.0    5.0
b    one    12.0   15.0
     two     8.0    0.0

In [74]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterating over groups

In [75]:
df

,key1,key2,data1,data2
0,a,one,5,11
1,a,two,11,5
2,b,one,12,15
3,b,two,8,0
4,a,one,9,16


In [76]:
# the groupby creates a series of tuples that can be unpacked into name and group
for name, group in df.groupby('key1'):
    print(name)
    print(group)
    print('----------------')

a
  key1 key2  data1  data2
0    a  one      5     11
1    a  two     11      5
4    a  one      9     16
----------------
b
  key1 key2  data1  data2
2    b  one     12     15
3    b  two      8      0
----------------


In [79]:
for name, group in df.groupby('key2'):
    print(name)
    print(group)
    print(group.sum())
    print('----------------')


one
  key1 key2  data1  data2
0    a  one      5     11
2    b  one     12     15
4    a  one      9     16
key1           aba
key2     oneoneone
data1           26
data2           42
dtype: object
----------------
two
  key1 key2  data1  data2
1    a  two     11      5
3    b  two      8      0
key1         ab
key2     twotwo
data1        19
data2         5
dtype: object
----------------
